In [18]:
import lineRenderWorldSetup as LR
import cv2
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
from glob import glob
from natsort import natsorted
import os
import _3dTo2dSetup as D3
from sklearn.metrics import precision_score, recall_score, confusion_matrix
from scipy.ndimage import zoom

In [19]:
# Load projections
path = r"model_uncertainty\docAnglesModel"
proj_path = natsorted(glob(os.path.join(path, "*.npy")))
box_dim = LR.BOX_DIM
real_box_dim = 301

#load 3D model
path_real = r"solid_400001-segmentation-solid_400001-label.nrrd"
real_box = D3.get3dFigure(real_box_dim, path_real)
real_box = np.where(zoom(np.copy(real_box),1.4) >= 1, 1, 0)

# order projection based on custom selection (see report)
print(proj_path)

['model_uncertainty\\docAnglesModel\\0_M30.7_0.7.npy', 'model_uncertainty\\docAnglesModel\\1_M11.0_20.0.npy', 'model_uncertainty\\docAnglesModel\\3_M-31.0_-0.1.npy', 'model_uncertainty\\docAnglesModel\\4_M-21.5_-24.8.npy', 'model_uncertainty\\docAnglesModel\\6_M-24.5_27.3.npy', 'model_uncertainty\\docAnglesModel\\7_M0.1_36.7.npy', 'model_uncertainty\\docAnglesModel\\8_M49.5_1.3.npy']


In [20]:
box_total_right = np.zeros([445,445,445])
box_total_left = np.zeros([445,445,445])
# select n projections to intersect in loop
for i in range(len(proj_path)): #len(proj_path)
  current_box = np.load(proj_path[i])
  if i > 1: # every sequence but 0 and 1 contain left coronary artery only
    box_total_left = box_total_left + current_box
  else: #2 first sequences contain right coronary artery only
    box_total_right = box_total_right + current_box

n_seqs_r = 2
n_seqs_l = 5
box_total_right = np.where(box_total_right == n_seqs_r , 1, 0)
box_total_left = np.where(box_total_left == n_seqs_l, 1, 0)

rx,ry,rz = LR.boxToWorldCorrds(box_total_right,box_dim)
lx,ly,lz = LR.boxToWorldCorrds(box_total_left,box_dim)

fx, fy, fz = D3.get3dBox(box_dim)

In [23]:
mlab.points3d(rx,ry,rz,
                  mode="cube", color=(1, 0, 0), scale_factor=1,opacity=0.01)
mlab.points3d(lx,ly,lz,
                  mode="cube", color=(0, 1, 0), scale_factor=1,opacity=0.01)
mlab.points3d(fx,fy,fz,
                  mode="cube", color=(1, 0, 0), scale_factor=1,opacity=0)
mlab.outline()
mlab.show()

: 